# 텐서플로를 사용해 배치 정규화 구현

In [1]:
"""
Module Import
"""
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from functools import partial

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
"""
Graph 설계 : X (input) - hidden1 - bn1 - hidden2 - bn2 - Logits
"""
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") # 첫번째 매개변수 : 데이터 타입
y = tf.placeholder(tf.int32, shape=(None), name="y")

training = tf.placeholder_with_default(False, shape=(), name="training") # 첫번쨰 매개변수 : 플래이스홀더의 기본 값 (훈련하는 동안에는 True, 이외에는 False로 설정)
my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9) # 래퍼함수 : 어떤 함수를 감싼 형태 ***

# 1. 첫번째 Hidden Layer : "tf.layers.dense" 활용하여 완전연결층 구축
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1") 

# 2. 배치 정규화 층
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1) # 활성화함수 elu 사용

# 3. 두번째 Hidden Layer
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")

# 4. 배치정규화 층
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

In [3]:
"""
Defining Shuffle_batch Function 
"""
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [4]:
"""
Set DataSet, to Training/Test/Valid
"""
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [5]:
"""
Loss 함수
"""
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [6]:
"""
Training : Loss 값을 최소화하여 가장 최적화된 training_op 산출, learning_rate = 0.01
"""
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
"""
평가 : 실제 y값을 잘 예측했는지 accuracy 계산
"""
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 40
batch_size = 50

with tf.Session() as8 sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                    feed_dict={training:True, X:X_batch, y: y_batch})
            accuracy_val = accuracy.eval(feed_dict = {X:X_valid, y:y_valid})
            print(epoch, "검증 세트 정확도:", accuracy_val)
            
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.0954
0 검증 세트 정확도: 0.127
0 검증 세트 정확도: 0.1676
0 검증 세트 정확도: 0.2042
0 검증 세트 정확도: 0.2486
0 검증 세트 정확도: 0.2784
0 검증 세트 정확도: 0.3
0 검증 세트 정확도: 0.3294
0 검증 세트 정확도: 0.3474
0 검증 세트 정확도: 0.3612
0 검증 세트 정확도: 0.3896
0 검증 세트 정확도: 0.415
0 검증 세트 정확도: 0.4378
0 검증 세트 정확도: 0.4596
0 검증 세트 정확도: 0.4796
0 검증 세트 정확도: 0.4966
0 검증 세트 정확도: 0.5134
0 검증 세트 정확도: 0.535
0 검증 세트 정확도: 0.5432
0 검증 세트 정확도: 0.564
0 검증 세트 정확도: 0.5824
0 검증 세트 정확도: 0.5894
0 검증 세트 정확도: 0.6078
0 검증 세트 정확도: 0.6168
0 검증 세트 정확도: 0.6308
0 검증 세트 정확도: 0.6426
0 검증 세트 정확도: 0.6582
0 검증 세트 정확도: 0.6652
0 검증 세트 정확도: 0.6724
0 검증 세트 정확도: 0.6798
0 검증 세트 정확도: 0.6868
0 검증 세트 정확도: 0.6924
0 검증 세트 정확도: 0.6968
0 검증 세트 정확도: 0.7036
0 검증 세트 정확도: 0.707
0 검증 세트 정확도: 0.715
0 검증 세트 정확도: 0.7192
0 검증 세트 정확도: 0.7198
0 검증 세트 정확도: 0.7244
0 검증 세트 정확도: 0.7258
0 검증 세트 정확도: 0.7326
0 검증 세트 정확도: 0.735
0 검증 세트 정확도: 0.7368
0 검증 세트 정확도: 0.7394
0 검증 세트 정확도: 0.7428
0 검증 세트 정확도: 0.7488
0 검증 세트 정확도: 0.7484
0 검증 세트 정확도: 0.7538
0 검증 세트 정확도: 0.7574
0 검증 세트 정확도: 0.76
0 검증 세트 정확도:

0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8938
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8938
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8936
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.893
0 검증 세트 정확도: 0.8944
0 검증 세트 정확도: 0.8954
0 검증 세트 정확도: 0.8964
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8934
0 검증 세트 정확도: 0.8944
0 검증 세트 정확도: 0.8956
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8954
0 검증 세트 정확도: 0.8942
0 검증 세트 정확도: 0.8926
0 검증 세트 정확도: 0.8928
0 검증 세트 정확도: 0.893
0 검증 세트 정확도: 0.8916
0 검증 세트 정확도: 0.8914
0 검증 세트 정확도: 0.8936
0 검증 세트 정확도: 0.892
0 검증 세트 정확도: 0.8922
0 검증 세트 정확도: 0.8934
0 검증 세트 정확도: 0.892
0 검증 세트 정확도: 0.8908
0 검증 세트 정확도: 0.8918
0 검증 세트 정확도: 0.891
0 검증 세트 정확도: 0.8914
0 검증 세트 정확도: 0.8922
0 검증 세트 정확도: 0.8928
0 검증 세트 정확도: 0.894
0 검증 세트 정확도: 0.8948
0 검증 세트 정확도: 0.8944
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.8952
0 검증 세트 정확도: 0.896
0 검증 세트 정확도: 0.8962
0 검증 세트 정확도: 0.8958
0 검증 세트 정확도: 0.8966
0 검증 세트 정확도: 0.8964
0 검증 세트 정확도: 0.896
0 검증 세트 정확도: 0.8976
0 검증 세트 정확도: 0.8964
0 검증 세트 정확도: 0.8976
0 검증 세트 정확도: 0.8976
0 검증 세트 정확도: 0.8982
0 검증 세트 

0 검증 세트 정확도: 0.9176
0 검증 세트 정확도: 0.9172
0 검증 세트 정확도: 0.918
0 검증 세트 정확도: 0.9174
0 검증 세트 정확도: 0.9182
0 검증 세트 정확도: 0.9186
0 검증 세트 정확도: 0.918
0 검증 세트 정확도: 0.918
0 검증 세트 정확도: 0.9174
0 검증 세트 정확도: 0.9176
0 검증 세트 정확도: 0.916
0 검증 세트 정확도: 0.9162
0 검증 세트 정확도: 0.917
0 검증 세트 정확도: 0.9184
0 검증 세트 정확도: 0.9194
0 검증 세트 정확도: 0.9194
0 검증 세트 정확도: 0.9184
0 검증 세트 정확도: 0.9184
0 검증 세트 정확도: 0.919
0 검증 세트 정확도: 0.9188
0 검증 세트 정확도: 0.9186
0 검증 세트 정확도: 0.9196
0 검증 세트 정확도: 0.9202
0 검증 세트 정확도: 0.92
0 검증 세트 정확도: 0.9202
0 검증 세트 정확도: 0.9208
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.9204
0 검증 세트 정확도: 0.9202
0 검증 세트 정확도: 0.92
0 검증 세트 정확도: 0.9202
0 검증 세트 정확도: 0.9202
0 검증 세트 정확도: 0.9206
0 검증 세트 정확도: 0.9198
0 검증 세트 정확도: 0.9192
0 검증 세트 정확도: 0.9192
0 검증 세트 정확도: 0.9214
0 검증 세트 정확도: 0.9208
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.9216
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.9208
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.9206
0 검증 세트 정확도: 0.9208
0 검증 세트 정확도: 0.9216
0 검증 세트 정확도: 0.9214
0 검증 세트 정확도: 0.921
0 검증 세트 정확도: 0.9218
0 검증 세트 정확도: 0.9

1 검증 세트 정확도: 0.9324
1 검증 세트 정확도: 0.9312
1 검증 세트 정확도: 0.9312
1 검증 세트 정확도: 0.9318
1 검증 세트 정확도: 0.9334
1 검증 세트 정확도: 0.9332
1 검증 세트 정확도: 0.9334
1 검증 세트 정확도: 0.9326
1 검증 세트 정확도: 0.9318
1 검증 세트 정확도: 0.9334
1 검증 세트 정확도: 0.9326
1 검증 세트 정확도: 0.933
1 검증 세트 정확도: 0.9338
1 검증 세트 정확도: 0.9344
1 검증 세트 정확도: 0.9332
1 검증 세트 정확도: 0.9316
1 검증 세트 정확도: 0.9314
1 검증 세트 정확도: 0.9318
1 검증 세트 정확도: 0.931
1 검증 세트 정확도: 0.9322
1 검증 세트 정확도: 0.9306
1 검증 세트 정확도: 0.929
1 검증 세트 정확도: 0.9298
1 검증 세트 정확도: 0.9308
1 검증 세트 정확도: 0.9302
1 검증 세트 정확도: 0.9306
1 검증 세트 정확도: 0.9306
1 검증 세트 정확도: 0.9306
1 검증 세트 정확도: 0.933
1 검증 세트 정확도: 0.9328
1 검증 세트 정확도: 0.9338
1 검증 세트 정확도: 0.9346
1 검증 세트 정확도: 0.9336
1 검증 세트 정확도: 0.9328
1 검증 세트 정확도: 0.9328
1 검증 세트 정확도: 0.9324
1 검증 세트 정확도: 0.93
1 검증 세트 정확도: 0.931
1 검증 세트 정확도: 0.9328
1 검증 세트 정확도: 0.933
1 검증 세트 정확도: 0.9334
1 검증 세트 정확도: 0.9338
1 검증 세트 정확도: 0.9338
1 검증 세트 정확도: 0.9328
1 검증 세트 정확도: 0.9332
1 검증 세트 정확도: 0.9344
1 검증 세트 정확도: 0.9342
1 검증 세트 정확도: 0.9324
1 검증 세트 정확도: 0.932
1 검증 세트 정확도: 0.933
1 검증 세트 정확

1 검증 세트 정확도: 0.9422
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.9404
1 검증 세트 정확도: 0.9398
1 검증 세트 정확도: 0.9394
1 검증 세트 정확도: 0.9398
1 검증 세트 정확도: 0.9398
1 검증 세트 정확도: 0.9406
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.94
1 검증 세트 정확도: 0.9414
1 검증 세트 정확도: 0.9394
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.9396
1 검증 세트 정확도: 0.9398
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.9394
1 검증 세트 정확도: 0.9394
1 검증 세트 정확도: 0.9394
1 검증 세트 정확도: 0.9404
1 검증 세트 정확도: 0.9392
1 검증 세트 정확도: 0.9388
1 검증 세트 정확도: 0.9392
1 검증 세트 정확도: 0.939
1 검증 세트 정확도: 0.9402
1 검증 세트 정확도: 0.94
1 검증 세트 정확도: 0.9404
1 검증 세트 정확도: 0.9408
1 검증 세트 정확도: 0.9404
1 검증 세트 정확도: 0.9412
1 검증 세트 정확도: 0.9418
1 검증 세트 정확도: 0.9408
1 검증 세트 정확도: 0.9412
1 검증 세트 정확도: 0.9408
1 검증 세트 정확도: 0.9408
1 검증 세트 정확도: 0.9398
1 검증 세트 정확도: 0.94
1 검증 세트 정확도: 0.9412
1 검증 세트 정확도: 0.9408
1 검증 세트 정확도: 0.942
1 검증 세트 정확도: 0.9414
1 검증 세트 정확도: 0.9422
1 검증 세트 정확도: 0.9418
1 검증 세트 정확도: 0.9418
1 검증 세트 정확도: 0.9432
1 검증 세트 정확도: 0.9422
1 검증 세트 정확도: 0.9428
1 검증 세트 정확도: 0.9426
1 검증 세트 정확도: 0.9426
1 검증 세트 

...

38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9828
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9828
38 검증 세트 정확도: 0.9828
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9834
38 검증 세트 정확도: 0.983
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9806
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.98
38 검증 세트 정확도: 0.9806
38 검증 세트 정확도: 0.9798
38 검증 세트 정확도: 0.9798
38 검증 세트 정확도: 0.9812
38 검증 세트 정확도: 0.9812
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9812
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9812
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9828
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9828
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.9824

38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.9806
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.9804
38 검증 세트 정확도: 0.9802
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9826
38 검증 세트 정확도: 0.9824
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.982
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9822
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9818
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.9802
38 검증 세트 정확도: 0.9796
38 검증 세트 정확도: 0.9806
38 검증 세트 정확도: 0.9816
38 검증 세트 정확도: 0.9812
38 검증 세트 정확도: 0.9814
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.9808
38 검증 세트 정확도: 0.981
38 검증 세트 정확도: 0.981
38 검

39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9814
39 검증 세트 정확도: 0.9808
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9838
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9836
39 검증 세트 정확도: 0.9838
39 검증 세트 정확도: 0.9834
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9838
39 검증 세트 정확도: 0.9838
39 검증 세트 정확도: 0.9838
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.9834
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9834
39 검증 세트 정확도: 0.9834
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.9828
3

39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9814
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9814
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.9802
39 검증 세트 정확도: 0.9806
39 검증 세트 정확도: 0.9804
39 검증 세트 정확도: 0.9808
39 검증 세트 정확도: 0.9814
39 검증 세트 정확도: 0.981
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9824
3

39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.9828
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9832
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.9834
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.9804
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9826
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.983
39 검증 세트 정확도: 0.982
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9812
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9822
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9824
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.9816
39 검증 세트 정확도: 0.9818
39 검증 세트 정확도: 0.981